In [1]:
from langchain_community.retrievers import BM25Retriever
from typing import List
import jieba  # 分词方法
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 循环字节分块
from langchain.vectorstores import FAISS, Qdrant  # 向量数据库
from langchain.document_loaders import TextLoader  # 加载文本的加载器
from langchain_huggingface import HuggingFaceEmbeddings  # type: ignore # 向量器

加载数据

In [2]:
loader = TextLoader("drug_100.txt")
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 每个分块的大小
    chunk_overlap=0,  # 分块之间的重叠部分
    length_function=len,  # 使用长度函数来计算分块大小
    separators=["\n"],  # 分块的分隔符
)
docs = text_splitter.split_documents(document)  # 分割文档为多个分块,以列表形式返回

In [3]:
docs[0]

Document(metadata={'source': 'drug_100.txt'}, page_content='"氨肽素\\n药品类型:血液系统药物，促进白细胞增生药\\n英文名:Ampeptide Elemente\\n剂型:片剂：0.2g\\n适应证:1.适用于治疗特发性血小板减少性紫癜(原发性血小板减少性紫癜)、过敏性紫癜、慢性白细胞减少症、慢性再生障碍性贫血。2.对银屑病也有一定疗效。\\n禁忌证:尚不明确。\\n注意事项:少数患者于停药后可能出现反跳现象，但继续用药仍可有效。\\n不良反应:个别病例服用后有腹部不适。\\n用法用量:口服给药：每次0.1g，每天3次。用药至少4周，有效者可连续服用。\\n药物相互作用:尚不明确。",\n"利可君\\n药品类型:血液系统药物，促进白细胞增生药\\n英文名:Leucogen\\n别名:利血生\\n剂型:片剂：10mg，20mg。\\n适应证:用于防治各种原因引起的白细胞减少、再生障碍性贫血等。\\n禁忌证:尚不明确。\\n注意事项:使用本药剂量要适当，过高或过低均影响疗效。\\n不良反应:本药无不良反应报道。\\n用法用量:口服给药：1.成人：每次10～20mg，每天3次，疗程1个月。2.儿童：每次10mg，每天2～3次，疗程1个月。\\n药物相互作用:尚不明确。",')

使用BM25方式检索

In [ ]:
def preprocessing_func(text: str) -> List[str]:
    """
    分词函数，将文本分词为列表
    """
    return list(jieba.cut(text))  # 使用jieba进行中文分词

bm25 = BM25Retriever(
    docs=docs, # 文档列表
    k = 5,  # 返回的文档数量
)
print(bm25.k) # 输出BM25检索器的k值

# 原始的bm25算法分词是使用的英文分词法, 所以需要去适配中文的分词策略,使用jieba进行中文分词
# from_documents()方式是用于->自动化处理文档的分词和索引构建​​
retriever = bm25.from_documents(
    docs,
    preprocess_func=preprocessing_func,  # 使用自定义的分词函数
    k=5,  # 返回的文档数量
)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


10


Loading model cost 0.948 seconds.
Prefix dict has been built successfully.


测试

In [5]:
retriever.invoke("内分泌药物有哪些, 作用是什么？")  # 调用检索器进行查询

[Document(metadata={'source': 'drug_100.txt'}, page_content='\n"去氨加压素\\n药品类型:内分泌系统药物，下丘脑及影响内分泌的药物\\n英文名:Desmopressin\\n别名:的斯加压素；弥凝鼻；Desmospray；Minirin\\n剂型:1.鼻喷雾剂（Desmospray）：500μg(5ml)；2.鼻滴入剂：200μg(2ml)；3.片剂：0.1mg，0.2mg；4.注射剂：0.4μg(1ml)。\\n适应证:1.轻中度血友病甲、血管性血友病1型和2型(2B型除外)。2.因尿毒症、肝硬化或药物诱发出血(如肝素、阿司匹林和右旋糖酐等)，其他血小板功能障碍和不明原因引起的出血时间过长。3.心脏手术引起的出血时间过长。4.其他手术引起的出血时间过长。不过，本品对重症血友病甲疗效较差。5.尿崩症，由于本品作用时间较长并可鼻腔或口服给药，因此是治疗尿崩症的首选药物。6.小儿遗尿。\\n禁忌证:1.对本品过敏者、哺乳者禁用。2.烦渴症、不稳定型心绞痛和严重充血性心衰患者禁用。3.禁用于2B型遗传性假性血友病患者。4.孕妇分娩前禁用；如有必要，分娩后立即给药。\\n注意事项:1.有血栓形成倾向、冠心病、高血压者慎用。2.治疗期间需限制饮水。3.老年人及幼儿、有颅内压增高趋势的患者及水电解质紊乱的患者慎用。4.需服利尿剂的患者、心功能不全、HB型血友病、不稳定性心绞痛等患者忌用。5.用药过量会增加水潴留和低钠血症的危险。对已出现低钠血症的患者，应停药、限制摄入量，伴有症状的患者，可输注等渗或高渗氯化钠注射剂。6.水钠潴留严重者可使用呋噻米利尿。\\n不良反应:不常见。1.偶有短暂头痛、恶心、轻度腹痛、血压升高和心率增快，减少剂量常可消失。2.少数患者肌内注射局部可出现疼痛。长期鼻黏膜给药可发生鼻黏膜充血、鼻炎、鼻黏膜萎缩，影响药物吸收。过量会引起水中毒和低钠血症。3.使用大剂量时，会产生头痛、一过性血压下降、反射性心率加速、面红、胃痛和恶心。\\n用法用量:1.中枢性尿崩症替代治疗：(1)喷雾或滴入制剂鼻腔给药：每次5～10μg，每天1～2次；(2)口服片剂给药：每次0.1～0.2mg(100～200μg)，每天3次；(3)肌内注射，每次1～4μg，每天1～2次。由于各个患者的病情轻重不同，用药应从小剂量开

In [6]:
from rank_bm25 import BM25Okapi
texts = [i.page_content for i in docs]  # 提取文档内容
texts_processed = [preprocessing_func(text) for text in texts]  # 对每个文档chunk进行分词处理,得到所有的词表(用于索引所有的单词)

# print(texts_processed)  # 输出第一个文档chunk的分词结果


In [7]:
vectorizer = BM25Okapi(texts_processed)  # 创建BM25向量化器
vectorizer.get_top_n(preprocessing_func("内分泌药物有哪些, 作用是什么？"), texts, n=5)  # 获取与查询最相关的前5个文档, 比较“骨折了怎么办”与texts的BM25相似度

['\n"去氨加压素\\n药品类型:内分泌系统药物，下丘脑及影响内分泌的药物\\n英文名:Desmopressin\\n别名:的斯加压素；弥凝鼻；Desmospray；Minirin\\n剂型:1.鼻喷雾剂（Desmospray）：500μg(5ml)；2.鼻滴入剂：200μg(2ml)；3.片剂：0.1mg，0.2mg；4.注射剂：0.4μg(1ml)。\\n适应证:1.轻中度血友病甲、血管性血友病1型和2型(2B型除外)。2.因尿毒症、肝硬化或药物诱发出血(如肝素、阿司匹林和右旋糖酐等)，其他血小板功能障碍和不明原因引起的出血时间过长。3.心脏手术引起的出血时间过长。4.其他手术引起的出血时间过长。不过，本品对重症血友病甲疗效较差。5.尿崩症，由于本品作用时间较长并可鼻腔或口服给药，因此是治疗尿崩症的首选药物。6.小儿遗尿。\\n禁忌证:1.对本品过敏者、哺乳者禁用。2.烦渴症、不稳定型心绞痛和严重充血性心衰患者禁用。3.禁用于2B型遗传性假性血友病患者。4.孕妇分娩前禁用；如有必要，分娩后立即给药。\\n注意事项:1.有血栓形成倾向、冠心病、高血压者慎用。2.治疗期间需限制饮水。3.老年人及幼儿、有颅内压增高趋势的患者及水电解质紊乱的患者慎用。4.需服利尿剂的患者、心功能不全、HB型血友病、不稳定性心绞痛等患者忌用。5.用药过量会增加水潴留和低钠血症的危险。对已出现低钠血症的患者，应停药、限制摄入量，伴有症状的患者，可输注等渗或高渗氯化钠注射剂。6.水钠潴留严重者可使用呋噻米利尿。\\n不良反应:不常见。1.偶有短暂头痛、恶心、轻度腹痛、血压升高和心率增快，减少剂量常可消失。2.少数患者肌内注射局部可出现疼痛。长期鼻黏膜给药可发生鼻黏膜充血、鼻炎、鼻黏膜萎缩，影响药物吸收。过量会引起水中毒和低钠血症。3.使用大剂量时，会产生头痛、一过性血压下降、反射性心率加速、面红、胃痛和恶心。\\n用法用量:1.中枢性尿崩症替代治疗：(1)喷雾或滴入制剂鼻腔给药：每次5～10μg，每天1～2次；(2)口服片剂给药：每次0.1～0.2mg(100～200μg)，每天3次；(3)肌内注射，每次1～4μg，每天1～2次。由于各个患者的病情轻重不同，用药应从小剂量开始，根据患者反应来调整药量。2.轻至中等的甲型血友病治疗，一般以0.3～0.4μg/kg(300～400ng/kg)进

使用embedding向量进行检索

In [8]:
from huggingface_hub import snapshot_download
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"   # 使用国内镜像
model_path = snapshot_download(repo_id="BAAI/bge-m3", repo_type="model")  # 下载模型

/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 89685.76it/s]


In [9]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cpu"},  # 由于显存不够, 只能将embeddings模型放在CPU上运行, 如果可以的话将device改为"cuda"可以使用GPU加速计算
)

需要提前部署Qdrant的api
```bash
wget https://github.com/qdrant/qdrant/releases/download/v1.14.1/qdrant-x86_64-unknown-linux-musl.tar.gz
tar -xzf qdrant-x86_64-unknown-linux-musl.tar.gz
./qdrant
```

In [10]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

try:
    collections = client.get_collections()  # 测试连接
    print("连接成功！", collections)
except Exception as e:
    print("连接失败:", e)

连接成功！ collections=[]


使用FAISS实现向量生成 

from_documents()方式是用于->自动化处理文档的分词和索引构建​​, docs是表示需要转换的文档, embeddings表示去使用的embeddings向量

In [11]:
db = Qdrant.from_documents(
    docs,
    embeddings,
)

In [38]:
bm25res = vectorizer.get_top_n(
    preprocessing_func("内分泌药物有哪些, 作用是什么？"),  # 查询文本
    texts,  # 文档列表
    n=5,  # 返回的文档数量
)
 
bm25res

['\n"去氨加压素\\n药品类型:内分泌系统药物，下丘脑及影响内分泌的药物\\n英文名:Desmopressin\\n别名:的斯加压素；弥凝鼻；Desmospray；Minirin\\n剂型:1.鼻喷雾剂（Desmospray）：500μg(5ml)；2.鼻滴入剂：200μg(2ml)；3.片剂：0.1mg，0.2mg；4.注射剂：0.4μg(1ml)。\\n适应证:1.轻中度血友病甲、血管性血友病1型和2型(2B型除外)。2.因尿毒症、肝硬化或药物诱发出血(如肝素、阿司匹林和右旋糖酐等)，其他血小板功能障碍和不明原因引起的出血时间过长。3.心脏手术引起的出血时间过长。4.其他手术引起的出血时间过长。不过，本品对重症血友病甲疗效较差。5.尿崩症，由于本品作用时间较长并可鼻腔或口服给药，因此是治疗尿崩症的首选药物。6.小儿遗尿。\\n禁忌证:1.对本品过敏者、哺乳者禁用。2.烦渴症、不稳定型心绞痛和严重充血性心衰患者禁用。3.禁用于2B型遗传性假性血友病患者。4.孕妇分娩前禁用；如有必要，分娩后立即给药。\\n注意事项:1.有血栓形成倾向、冠心病、高血压者慎用。2.治疗期间需限制饮水。3.老年人及幼儿、有颅内压增高趋势的患者及水电解质紊乱的患者慎用。4.需服利尿剂的患者、心功能不全、HB型血友病、不稳定性心绞痛等患者忌用。5.用药过量会增加水潴留和低钠血症的危险。对已出现低钠血症的患者，应停药、限制摄入量，伴有症状的患者，可输注等渗或高渗氯化钠注射剂。6.水钠潴留严重者可使用呋噻米利尿。\\n不良反应:不常见。1.偶有短暂头痛、恶心、轻度腹痛、血压升高和心率增快，减少剂量常可消失。2.少数患者肌内注射局部可出现疼痛。长期鼻黏膜给药可发生鼻黏膜充血、鼻炎、鼻黏膜萎缩，影响药物吸收。过量会引起水中毒和低钠血症。3.使用大剂量时，会产生头痛、一过性血压下降、反射性心率加速、面红、胃痛和恶心。\\n用法用量:1.中枢性尿崩症替代治疗：(1)喷雾或滴入制剂鼻腔给药：每次5～10μg，每天1～2次；(2)口服片剂给药：每次0.1～0.2mg(100～200μg)，每天3次；(3)肌内注射，每次1～4μg，每天1～2次。由于各个患者的病情轻重不同，用药应从小剂量开始，根据患者反应来调整药量。2.轻至中等的甲型血友病治疗，一般以0.3～0.4μg/kg(300～400ng/kg)进

In [39]:
vector_res = db.similarity_search(
    "内分泌药物有哪些, 作用是什么？",  # 查询文本
    k=5,  # 返回的文档数量
)
vector_res


[Document(metadata={'source': 'drug_100.txt', '_id': '1a5bac48-d303-47ce-ba46-746215f4244f', '_collection_name': 'f85c5a1ab31c4d02ba08b1d1c35945e9'}, page_content='"氨肽素\\n药品类型:血液系统药物，促进白细胞增生药\\n英文名:Ampeptide Elemente\\n剂型:片剂：0.2g\\n适应证:1.适用于治疗特发性血小板减少性紫癜(原发性血小板减少性紫癜)、过敏性紫癜、慢性白细胞减少症、慢性再生障碍性贫血。2.对银屑病也有一定疗效。\\n禁忌证:尚不明确。\\n注意事项:少数患者于停药后可能出现反跳现象，但继续用药仍可有效。\\n不良反应:个别病例服用后有腹部不适。\\n用法用量:口服给药：每次0.1g，每天3次。用药至少4周，有效者可连续服用。\\n药物相互作用:尚不明确。",\n"利可君\\n药品类型:血液系统药物，促进白细胞增生药\\n英文名:Leucogen\\n别名:利血生\\n剂型:片剂：10mg，20mg。\\n适应证:用于防治各种原因引起的白细胞减少、再生障碍性贫血等。\\n禁忌证:尚不明确。\\n注意事项:使用本药剂量要适当，过高或过低均影响疗效。\\n不良反应:本药无不良反应报道。\\n用法用量:口服给药：1.成人：每次10～20mg，每天3次，疗程1个月。2.儿童：每次10mg，每天2～3次，疗程1个月。\\n药物相互作用:尚不明确。",'),
 Document(metadata={'source': 'drug_100.txt', '_id': 'e05d215c-d5a1-4a8c-be0b-62396268f1eb', '_collection_name': 'f85c5a1ab31c4d02ba08b1d1c35945e9'}, page_content='\n"丙酸睾酮\\n药品类型:内分泌系统药物，性腺疾病用药\\n英文名:Testosterone Propionate\\n别名:丙酸睾丸素；丙酸睾丸酮；苯丙酸酯苯乙酸睾酮；睾酮丙酸酯；Andronate\\n剂型:注射剂：10mg(1ml)，25mg(1ml)，50mg(1ml)，100mg(1ml)。\\

对文本召回和向量召回的方法进行**重排序**

In [40]:
def rrf(vector_result: List[str], bm25_result: List[str], k: int = 10, m: int = 60) -> List[str]:
    """
    RRF (Reciprocal Rank Fusion) 算法融合向量检索和BM25检索结果
    param:
        vector_result: 向量检索结果列表
        bm25_result: BM25检索结果列表
        k: 排序后返回前k个文档
        m: 用于倒数排名加权的参数
    """
    # 创建一个字典来存储每个文档的分数
    doc_scores = {}

    # 遍历两组结果, 计算每个文档的融合得分
    for rank, doc_id in enumerate(vector_result):
        doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank + m)  # 向量检索结果的倒数排名加权
    
    for rank, doc_id in enumerate(bm25_result):
        doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank + m)  # BM25检索结果的倒数排名加权
    
    # 按照分数排序并返回前k个文档
    sorted_result = [d for d, _ in sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)[:k]]

    return sorted_result  # 返回前k个文档内容


In [41]:
vector_result = [i.page_content for i in vector_res]  # 提取向量检索结果的文档内容
text_result = [i for i in bm25res]  # 提取BM25检索结果的文档内容

rrf_result = rrf(vector_result, text_result, k=5, m=60)  # 使用RRF算法融合结果
rrf_result

['\n"去氨加压素\\n药品类型:内分泌系统药物，下丘脑及影响内分泌的药物\\n英文名:Desmopressin\\n别名:的斯加压素；弥凝鼻；Desmospray；Minirin\\n剂型:1.鼻喷雾剂（Desmospray）：500μg(5ml)；2.鼻滴入剂：200μg(2ml)；3.片剂：0.1mg，0.2mg；4.注射剂：0.4μg(1ml)。\\n适应证:1.轻中度血友病甲、血管性血友病1型和2型(2B型除外)。2.因尿毒症、肝硬化或药物诱发出血(如肝素、阿司匹林和右旋糖酐等)，其他血小板功能障碍和不明原因引起的出血时间过长。3.心脏手术引起的出血时间过长。4.其他手术引起的出血时间过长。不过，本品对重症血友病甲疗效较差。5.尿崩症，由于本品作用时间较长并可鼻腔或口服给药，因此是治疗尿崩症的首选药物。6.小儿遗尿。\\n禁忌证:1.对本品过敏者、哺乳者禁用。2.烦渴症、不稳定型心绞痛和严重充血性心衰患者禁用。3.禁用于2B型遗传性假性血友病患者。4.孕妇分娩前禁用；如有必要，分娩后立即给药。\\n注意事项:1.有血栓形成倾向、冠心病、高血压者慎用。2.治疗期间需限制饮水。3.老年人及幼儿、有颅内压增高趋势的患者及水电解质紊乱的患者慎用。4.需服利尿剂的患者、心功能不全、HB型血友病、不稳定性心绞痛等患者忌用。5.用药过量会增加水潴留和低钠血症的危险。对已出现低钠血症的患者，应停药、限制摄入量，伴有症状的患者，可输注等渗或高渗氯化钠注射剂。6.水钠潴留严重者可使用呋噻米利尿。\\n不良反应:不常见。1.偶有短暂头痛、恶心、轻度腹痛、血压升高和心率增快，减少剂量常可消失。2.少数患者肌内注射局部可出现疼痛。长期鼻黏膜给药可发生鼻黏膜充血、鼻炎、鼻黏膜萎缩，影响药物吸收。过量会引起水中毒和低钠血症。3.使用大剂量时，会产生头痛、一过性血压下降、反射性心率加速、面红、胃痛和恶心。\\n用法用量:1.中枢性尿崩症替代治疗：(1)喷雾或滴入制剂鼻腔给药：每次5～10μg，每天1～2次；(2)口服片剂给药：每次0.1～0.2mg(100～200μg)，每天3次；(3)肌内注射，每次1～4μg，每天1～2次。由于各个患者的病情轻重不同，用药应从小剂量开始，根据患者反应来调整药量。2.轻至中等的甲型血友病治疗，一般以0.3～0.4μg/kg(300～400ng/kg)进

使用LLM + RAG的方式进行召回

In [42]:
prompt = '''
    任务目标：根据检索出的文档回答用户问题
    任务要求：
    1. 只使用检索出的文档内容回答问题
    2. 不要添加任何额外的内容
    3. 如果检索结果中没有相关信息，请回答“无法回答”

用户问题:
{}

检索的文档:
{}

'''

In [43]:
from langchain_openai import ChatOpenAI  # OpenAI的聊天模型
model = ChatOpenAI(
    model="Qwen3-8B",
    base_url="http://localhost:8000/v1",  # 本地部署的Qwen3模型地址
    api_key="your_api_key",  # 替换为你的API密钥,
    max_tokens=1000,  # 最大生成的token数量
)

In [48]:
prompt.format(
        "内分泌药物有哪些, 作用是什么？",  # 用户问题
        "\n".join(rrf_result)  # 将融合后的文档内容作为检索结果传入
    )

'\n    任务目标：根据检索出的文档回答用户问题\n    任务要求：\n    1. 只使用检索出的文档内容回答问题\n    2. 不要添加任何额外的内容\n    3. 如果检索结果中没有相关信息，请回答“无法回答”\n\n用户问题:\n内分泌药物有哪些, 作用是什么？\n\n检索的文档:\n\n"去氨加压素\\n药品类型:内分泌系统药物，下丘脑及影响内分泌的药物\\n英文名:Desmopressin\\n别名:的斯加压素；弥凝鼻；Desmospray；Minirin\\n剂型:1.鼻喷雾剂（Desmospray）：500μg(5ml)；2.鼻滴入剂：200μg(2ml)；3.片剂：0.1mg，0.2mg；4.注射剂：0.4μg(1ml)。\\n适应证:1.轻中度血友病甲、血管性血友病1型和2型(2B型除外)。2.因尿毒症、肝硬化或药物诱发出血(如肝素、阿司匹林和右旋糖酐等)，其他血小板功能障碍和不明原因引起的出血时间过长。3.心脏手术引起的出血时间过长。4.其他手术引起的出血时间过长。不过，本品对重症血友病甲疗效较差。5.尿崩症，由于本品作用时间较长并可鼻腔或口服给药，因此是治疗尿崩症的首选药物。6.小儿遗尿。\\n禁忌证:1.对本品过敏者、哺乳者禁用。2.烦渴症、不稳定型心绞痛和严重充血性心衰患者禁用。3.禁用于2B型遗传性假性血友病患者。4.孕妇分娩前禁用；如有必要，分娩后立即给药。\\n注意事项:1.有血栓形成倾向、冠心病、高血压者慎用。2.治疗期间需限制饮水。3.老年人及幼儿、有颅内压增高趋势的患者及水电解质紊乱的患者慎用。4.需服利尿剂的患者、心功能不全、HB型血友病、不稳定性心绞痛等患者忌用。5.用药过量会增加水潴留和低钠血症的危险。对已出现低钠血症的患者，应停药、限制摄入量，伴有症状的患者，可输注等渗或高渗氯化钠注射剂。6.水钠潴留严重者可使用呋噻米利尿。\\n不良反应:不常见。1.偶有短暂头痛、恶心、轻度腹痛、血压升高和心率增快，减少剂量常可消失。2.少数患者肌内注射局部可出现疼痛。长期鼻黏膜给药可发生鼻黏膜充血、鼻炎、鼻黏膜萎缩，影响药物吸收。过量会引起水中毒和低钠血症。3.使用大剂量时，会产生头痛、一过性血压下降、反射性心率加速、面红、胃痛和恶心。\\n用法用量:1.中枢性尿崩症替代治疗：(1)喷雾或滴入制剂鼻腔给药：每次5～10μg

采用RAG检索增强生成的模型回答

In [49]:
res = model.invoke(
    prompt.format(
        "内分泌药物有哪些, 作用是什么？",  # 用户问题
        "\n".join(rrf_result)  # 将融合后的文档内容作为检索结果传入
    )
)

print(res.content)  # 输出模型的回答结果



内分泌药物包括以下几种及其作用：  
1. **去氨加压素**：用于治疗轻中度血友病甲、血管性血友病、尿崩症、小儿遗尿等，通过调节抗利尿激素作用减少出血和控制尿量。  
2. **十一酸睾酮**：用于原发性或继发性睾丸功能减退、男性性功能低下、再生障碍性贫血等，补充雄激素以改善性腺功能及相关症状。  
3. **丙酸睾酮**：用于男性性功能降低、绝经期乳腺癌、青春期延迟、老年性骨质疏松等，通过补充睾酮调节内分泌及代谢功能。  
4. **羟苯磺酸钙**：用于糖尿病性视网膜病变、周围循环障碍（如心绞痛、脑血栓）、静脉曲张综合征等，改善微循环和血液流变学异常。  

（以上内容均基于检索文档信息）


原始模型的回答

In [51]:
res = model.invoke("内分泌药物有哪些, 作用是什么？")
print(res.content)  # 输出模型的回答结果



内分泌药物主要用于调节人体内分泌系统功能，治疗由激素分泌异常或代谢紊乱引起的疾病。以下是常见的内分泌药物分类及其作用机制和用途：

---

### **一、糖尿病相关药物**
1. **胰岛素类药物**  
   - **作用**：直接补充胰岛素，调节血糖。  
   - **类型**：  
     - **速效/短效**（如门冬胰岛素）：餐前注射，控制餐后血糖。  
     - **中效/长效**（如甘精胰岛素）：全天维持基础血糖水平。  
     - **预混胰岛素**（如门冬胰岛素30）：结合速效和中效，简化注射方案。  
   - **用途**：1型糖尿病、2型糖尿病（胰岛素分泌不足或抵抗）、妊娠糖尿病等。

2. **口服降糖药**  
   - **胰岛素促泌剂**（如格列美脲）：刺激胰岛β细胞分泌胰岛素。  
   - **双胍类**（如二甲双胍）：减少肝糖生成，提高胰岛素敏感性。  
   - **α-葡萄糖苷酶抑制剂**（如阿卡波糖）：延缓碳水化合物吸收，降低餐后血糖。  
   - **DPP-4抑制剂**（如西格列汀）：增加内源性GLP-1水平，促进胰岛素分泌。  
   - **SGLT2抑制剂**（如恩格列净）：通过肾脏排泄葡萄糖，降低血糖。  
   -
